In [2]:
# REDO SCRAPE TO MAKE youtube_stats.csv



In [3]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import matplotlib.pyplot as plt
# import ytcreds
import json
import credentials


DEVELOPER_KEY = credentials.API_KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

In [4]:
#use youtube.search().list(part='id, snippet')

query="why I quit my job"
# token="CDIQAA"
order='relevance'
max_results=2000
location=None
location_radius=None

# videos_list=[]
responses_list =[]

for i in range(0,max_results,50):
    print(i)
    if i ==0:
        next_token=None
        prev_token = None
    search_response = youtube.search().list(
        q=query,
        type="video",
        pageToken=next_token,
        order = order,
        part="id,snippet",
        maxResults=max_results,
        location=location,
        locationRadius=location_radius).execute()
    
    responses_list.append(search_response)
#     print(search_response.keys())
    
    
    #GET NEXT PAGE OF RESULTS
    if 'nextPageToken' in search_response.keys():
        next_token=search_response['nextPageToken']
    else:
        next_token=None
        
    if 'prevPageToken' in search_response.keys():
        prev_token=search_response['prevPageToken']
    else:
        prev_token=None
        
    
#     print("NEXT:", next_token)
#     print("PREV:",prev_token)

    print("----")

    
    print(responses_list[-1])

0
----
{'kind': 'youtube#searchListResponse', 'etag': '_8kUiQ8lFBmcxxrymuvgGkxpBrU', 'nextPageToken': 'CDIQAA', 'regionCode': 'CA', 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'iGqkNBPSEB3bWicbSrS8idBKByM', 'id': {'kind': 'youtube#video', 'videoId': 'SZEo1KFjTn4'}, 'snippet': {'publishedAt': '2019-05-24T14:54:14Z', 'channelId': 'UCI_2HqC2a22Q4Jx41o9g72w', 'title': 'Should You QUIT Your Job? - The Most Life Changing Speech Ever (ft. Garyvee, Joe Rogan)', 'description': 'Should you quit your job and start a business? Joe Rogan & Garyvee talks about how to know when to quit your job and how to ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/SZEo1KFjTn4/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/SZEo1KFjTn4/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/SZEo1KFjTn4/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 

In [29]:
# responses_list[0]['items'][0]

In [30]:

# for response in responses_list:
#     print(response.keys())
#     for key in response.keys():

#         print(response['items'])
#         if key =="items":
#             item_list = response['items']
#             for item in item_list:
#                 video_dict = {}
#     #             print(item)
#                 title=item['snippet']['title']

#                 desc = item['snippet']['description']
#                 video_id = item['id']['videoId']
#                 channel_id = items['snippet']['channelId']
#                 print(desc)

#         print("____")

In [64]:
#create list of ids to iterate through - remove the layer of lists given for individual queries of youtube.search()

video_list = []
for response in responses_list:
#     print(response.keys())
    videos_list = response['items']
    for item in videos_list:
        
        video_title=item['snippet']['title']

#         desc = item['snippet']['description'] #only gives snippet of desc - add when getting transcripts
        video_id = item['id']['videoId']
        channel_id = item['snippet']['channelId']
        published_at = item['snippet']['publishedAt']
        video_dict = {
            "video_id": video_id,
            "video_title": video_title,
            "channel_id": channel_id,
            'publish_at': published_at
        }
        
        video_list.append(video_dict)
len(video_list)
        

1987

In [69]:
title = []
channelId = []
channelTitle = []
categoryId = []
videoId = []
viewCount = []
likeCount = []
dislikeCount = []
commentCount = []
favoriteCount = []
category = []
tags = []
videos = []
descriptions=[]
i = 0
for video in video_list:
    if i % 100 == 0:
        print("VIDEOS PROCESSED: ", i)
    videoId.append(video['video_id'])
    title.append(video['video_title'])
    stats = youtube.videos().list(
        part='statistics, snippet',
        id=video['video_id']).execute()
    channelId.append(stats['items'][0]['snippet']['channelId']) 
    channelTitle.append(stats['items'][0]['snippet']['channelTitle']) 
    categoryId.append(stats['items'][0]['snippet']['categoryId']) 
    favoriteCount.append(stats['items'][0]['statistics']['favoriteCount'])
    try:
        viewCount.append(stats['items'][0]['statistics']['viewCount']) 
    except:
        viewCount.append("Not available") 
    #Not every video has likes/dislikes enabled so they won't appear in JSON response
    try:
        likeCount.append(stats['items'][0]['statistics']['likeCount'])
    except:
   #Good to be aware of Channels that turn off their Likes
#         print("Video titled {0}, on Channel {1} Likes Count is not available".format(stats['items'][0]['snippet']['title'],
#                                                                                      stats['items'][0]['snippet']['channelTitle']))
        print(stats['items'][0]['statistics'].keys())
    #Appends "Not Available" to keep dictionary values aligned
        likeCount.append("Not available")

    try:
        dislikeCount.append(stats['items'][0]['statistics']['dislikeCount'])     
    except:
        #Good to be aware of Channels that turn off their Likes
#         print("Video titled {0}, on Channel {1} Dislikes Count is not available".format(stats['items'][0]['snippet']['title'],
#                                                                                         stats['items'][0]['snippet']['channelTitle']))
#         print(stats['items'][0]['statistics'].keys())
        dislikeCount.append("Not available")

    if 'commentCount' in stats['items'][0]['statistics'].keys():
        commentCount.append(stats['items'][0]['statistics']['commentCount'])
    else:
        commentCount.append(0)

    if 'tags' in stats['items'][0]['snippet'].keys():
        tags.append(stats['items'][0]['snippet']['tags'])
    else:
        #I'm not a fan of empty fields
        tags.append("No Tags")
        
    if 'description' in stats['items'][0]['snippet'].keys():
        descriptions.append(stats['items'][0]['snippet']['description'])
    else:
        #I'm not a fan of empty fields
        tags.append("No Description")
    
    #a given video iss equivelant to stats['items'][0]
    i += 1

VIDEOS PROCESSED:  0
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
VIDEOS PROCESSED:  100
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
VIDEOS PROCESSED:  200
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
VIDEOS PROCESSED:  300
VIDEOS PROCESSED:  400
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
VIDEOS PROCESSED:  500
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
VIDEOS PROCESSED:  600
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
VIDEOS PROCESSED:  700
VIDEOS PROCESSED:  800
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
dict_keys(['viewCount', 'favoriteCount', 'commentCount'])
VIDEOS PROCESSED:  900
VIDEOS PROCESSED:  1000
dict_keys(['viewCount', 'favoriteCount', 'commentCou

In [51]:
# stats

In [70]:
youtube_dict = {'tags':tags,'channelId': channelId,'channelTitle': channelTitle,
                'categoryId':categoryId,'title':title,'videoId':videoId,
                'viewCount':viewCount,'likeCount':likeCount,'dislikeCount':dislikeCount,
                'commentCount':commentCount,'favoriteCount':favoriteCount,
               "description": descriptions}

for key in youtube_dict.keys():
    print(key)
    print("LENGTH: ", len(youtube_dict[key]))
    print(youtube_dict[key][:5])
    
    print("---")


tags
LENGTH:  1987
[['should you quit your job', 'should you quit your job to start a business', 'signs you should quit your job', 'should i quit my job', 'should i quit my job to start my own business', 'joe rogan quit your job', 'quit your job', 'quit your job motivation', 'quitting my job', 'quit your job and do what you love', 'quit your job garyvee', 'quit your job joe rogan', 'quit your job if you hate it', 'how to know if you should quit your job', 'gary vee quit your job', 'when is the right time to quit your job'], 'No Tags', ['personal finance', 'quit your job', 'passive income', 'quit my job', 'why i quit my job for youtube', 'lessons about money', 'quit my job for youtube', 'passive income 2021', 'why i quit', 'quit job', 'big law', 'i quit my job', 'passive income online 2021', 'i quit my job in 2021', 'i quit my job in 2020', 'should i quit my job', 'quit my job to start a business', 'should i quit my job if it makes me unhappy', 'should i quit my job without another line

In [71]:
filename="youtube_stats.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(youtube_dict, f, ensure_ascii=False, indent=4)

In [17]:
#use youtube.videos().list(part='statistics, snippet')

"After 6 years, I've finally decided to leave the legal industry for good. In this video I'll be sharing why I quit my six-figure job as a ..."